In [1]:
import pandas as pd
from linearmodels import PooledOLS, PanelOLS, RandomEffects
import numpy as np
from scipy import stats


C:\Users\adamh\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data = pd.read_excel('panel_assignment.xlsx')

### ret = alpha + (average excess ret)*beta

### factor payoff -- time series from cross sectional regression
### beta: factor loading
### factor payof = 0?

In [3]:
data.head()
#long form

,firm_ident,return,beta,year
0,1,-0.004813,1.501050,1996
1,2,NaN,0.771553,1996
2,3,-0.004813,1.116419,1996
3,4,-0.004813,1.128749,1996
4,5,-0.001599,1.244929,1996


In [4]:
data=data.rename(columns={'return': 'ret'})

In [5]:
#data summary
data['ret'].describe()

count    24091.000000
mean        -0.001546
std          0.038328
min         -1.005126
25%         -0.004813
50%         -0.003933
75%         -0.002967
max          0.706354
Name: ret, dtype: float64

In [6]:
data['beta'].describe()
#beta should be 1

count    9073.000000
mean        1.104948
std         0.203569
min         0.660871
25%         0.940652
50%         1.081632
75%         1.248908
max         1.611615
Name: beta, dtype: float64

In [7]:
#drop missing nan values
data = data.dropna()

In [8]:
#multi-index
data = data.set_index(['firm_ident', 'year'])

In [9]:
#simple-Pooled Regression
model = PooledOLS.from_formula('ret ~ 1 + beta', data)
pooled = model.fit()
print(pooled)

                          PooledOLS Estimation Summary                          
Dep. Variable:                    ret   R-squared:                     3.118e-06
Estimator:                  PooledOLS   R-squared (Between):             -0.0087
No. Observations:                8856   R-squared (Within):           -9.129e-05
Date:                Thu, Nov 24 2022   R-squared (Overall):           3.118e-06
Time:                        08:43:11   Log-likelihood                 1.357e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      0.0276
Entities:                        1734   P-value                           0.8680
Avg Obs:                       5.1073   Distribution:                  F(1,8854)
Min Obs:                       1.0000                                           
Max Obs:                       11.000   F-statistic (robust):             0.0276
                            

In [10]:
#panel Regression with Fiexed Effects
#simple-Pooled Regression
model = PanelOLS.from_formula('ret ~ 1 + beta + EntityEffects', data)
fixed = model.fit(cov_type = 'clustered', cluster_entity= True) #across individuals
print(fixed)

                          PanelOLS Estimation Summary                           
Dep. Variable:                    ret   R-squared:                        0.0012
Estimator:                   PanelOLS   R-squared (Between):             -0.0119
No. Observations:                8856   R-squared (Within):               0.0012
Date:                Thu, Nov 24 2022   R-squared (Overall):             -0.0023
Time:                        08:43:11   Log-likelihood                  1.48e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      8.3576
Entities:                        1734   P-value                           0.0039
Avg Obs:                       5.1073   Distribution:                  F(1,7121)
Min Obs:                       1.0000                                           
Max Obs:                       11.000   F-statistic (robust):             5.0009
                            

In [11]:
#F test for fixed effexts verses OLS
print(fixed.f_pooled)

Pooled F-statistic
H0: Effects are zero
Statistic: 1.3111
P-value: 0.0000
Distributed: F(1733,7121)


Pooled --> one intercept

Fixed effects --> everybody gets thier own intercept

Random effects --> one distribution for the intercept


In [12]:
#Panel Regression with Random Effects
model = RandomEffects.from_formula('ret ~ 1 + beta', data)
random = model.fit()
print(random)



                        RandomEffects Estimation Summary                        
Dep. Variable:                    ret   R-squared:                     7.135e-05
Estimator:              RandomEffects   R-squared (Between):             -0.0030
No. Observations:                8856   R-squared (Within):               0.0008
Date:                Thu, Nov 24 2022   R-squared (Overall):             -0.0019
Time:                        08:43:12   Log-likelihood                 1.412e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      0.6318
Entities:                        1734   P-value                           0.4267
Avg Obs:                       5.1073   Distribution:                  F(1,8854)
Min Obs:                       1.0000                                           
Max Obs:                       11.000   F-statistic (robust):             2.8030
                            